In [2]:
import pandas as pd
import numpy as np
import seaborn
import time
import glob
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
import geopandas
from xgboost import XGBRegressor
from sklearn.feature_selection import RFE
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from numpy import absolute
from sklearn.model_selection import RandomizedSearchCV
import pymannkendall as mk
import geopandas as gpd
import warnings
warnings.filterwarnings("ignore")

In [2]:
def mergeCSV(path):
    all_filenames = glob.glob(path + "/*.csv")
    merge = pd.concat([pd.read_csv(f) for f in all_filenames ])
    return merge

def pull_clean (data):
    data =data.rename(columns={"Blue": "blue", "system:index": "LS_ID", "TIR1":"LS_Temp" })
    
    data = data.dropna(subset=['blue'])
    
    data[['sat', 'pathrow', 'date', 'junk']]  = data['LS_ID'].str.split('L', n = 1, expand = True)[1].str.split('_', n =4 , expand = True)
    data = data.drop(columns=['junk'])
    data['sat'] = 'L'+data['sat'] 
    data["date"] = pd.to_datetime(data["date"]).dt.date
    data["month"] = pd.to_datetime(data["date"]).dt.month
    data["year"] = pd.to_datetime(data["date"]).dt.year

    data['ST_QA'] =  data['ST_QA'] * 0.01
    data['ST_CDIST'] =  data['ST_CDIST'] * 0.01
    data = data[(data.ST_CDIST > 0.1) & (data.ST_QA < 273.15)]
    data['LS_Temp'] = data['LS_Temp']-273.15
    data = data[(data.LS_Temp > 0) & (data.LS_Temp < 35)]
    data['key'] = data['p_ID'].astype(str)+data['month'].astype(str)
    

    threshold = data.groupby(['p_ID','month'])['LS_Temp'].describe()[['mean','max', 'min', '25%', '75%']].reset_index()
    
    threshold['key'] = threshold ['p_ID'].astype(str)+threshold ['month'].astype(str)
    threshold ['iqr'] = threshold ['75%'] - threshold ['25%']
    threshold ['iqr_upper_bound'] = threshold['75%'] + 1.5*threshold ['iqr']
    threshold ['iqr_lower_bound'] = threshold['25%'] - 1.5*threshold ['iqr']
    threshold = threshold[['key', 'iqr_upper_bound', 'iqr_lower_bound']]
    
    #threshold['key'] = threshold['key'].astype(str)
    #data['key'] = data['key'].astype(str)
    data = data.merge(threshold, on='key', how='left')
    data = data[(data.LS_Temp < data.iqr_upper_bound) & (data.LS_Temp > data.iqr_lower_bound)]
 

    #data = data[['key','LS_ID','pathrow','sat','blue', 'green', 'nir', 'red', 'swir1', 'swir2', 'LS_Temp','CLOUD_COVER','ST_CDIST', 'ST_DRAD', 'ST_EMIS', 'ST_EMSD', 'ST_QA', 'ST_TRAD', 'ST_URAD', 
    #    'SUN_AZIMUTH', 'SUN_ELEVATION', 'date', 'year', 'month', 'ID_NUMBER', 'p_ID']]
    data=data[~(data.sat=='LC09')]
    #data = data[['LS_ID','pathrow','sat','blue', 'LS_Temp','ST_CDIST', 'ST_QA', 'date', 'year', 'month', 'ID_NUMBER', 'p_ID']]

    return data

def mktest(x):
    result = mk.original_test(x)
    return pd.Series(result, index=result._fields)

def seasonal_mktest(x):
    result = mk.seasonal_test(x)
    return pd.Series(result, index=result._fields)

In [6]:
path = r"D:\Estuary Temperature\global sr\Estuary_collection2"
sr = mergeCSV(path)
#sr = sr.drop_duplicates()
sr.to_csv("Global_estuary_sr_clean_final_raw.csv")

In [23]:
sr = pd.read_csv("Global_estuary_sr_clean_final_raw.csv")
sr = pull_clean (sr)
sr.to_csv("Global_estuary_sr_clean_final.csv", index = False)

In [ ]:
sr = sr[(sr.ST_CDIST > 0.2)]#0.2 is good
sr['Corrected_LS_Temp'] = 0.340+(0.967*sr['LS_Temp'])
sr = sr.drop_duplicates()
sr.to_csv("Global_estuary_sr_clean_final_v2.csv", index = False)

In [3]:
sr = pd.read_csv("Global_estuary_sr_clean_final_v2.csv")
sr['Blue']=sr['blue']
sr[['LS_ID', 'Aerosol', 'Blue', 'Green', 'Red', 'Nir','NirSD','Swir1', 'Swir2','CLOUD_COVER','ST_CDIST',
    'ST_DRAD', 'ST_EMIS', 'ST_EMSD','SUN_AZIMUTH','SUN_ELEVATION',
    'LS_Temp','Corrected_LS_Temp', 'sat', 'pathrow', 'date', 'month', 'year',
    'CONTINENT', 'COUNTRY', 'p_ID','ID_NUMBER']].to_csv("Global_estuary_sr_clean_final_v3.csv", index=False)

In [3]:
sr = pd.read_csv(r"D:\Estuary Temperature\global sr\final\Global_estuary_sr_clean_final_v3.csv")
# Count the number of estuary detected by landsat
print("Number of Estuary",len(sr['ID_NUMBER'].unique()))
print("Number of sites",len(sr['p_ID'].unique()))
print("year min", sr['year'].min())
print("year min", sr['year'].max())
print(sr.shape)

Number of Estuary 1060
Number of sites 195566
year min 1985
year min 2022
(64375639, 27)


In [4]:
#Count how many points per seasonal temperature per estuary
estuary_shp = geopandas.read_file(r"D:\Estuary Temperature\global sr\final\shp\estuary_centroid.shp")
def find_season(month, hemisphere):
    if hemisphere == 'Southern':
        season_month_south = {
            12:'Summer', 1:'Summer', 2:'Summer',
            3:'Autumn', 4:'Autumn', 5:'Autumn',
            6:'Winter', 7:'Winter', 8:'Winter',
            9:'Spring', 10:'Spring', 11:'Spring'}
        return season_month_south.get(month)
        
    elif hemisphere == 'Northern':
        season_month_north = {
            12:'Winter', 1:'Winter', 2:'Winter',
            3:'Spring', 4:'Spring', 5:'Spring',
            6:'Summer', 7:'Summer', 8:'Summer',
            9:'Autumn', 10:'Autumn', 11:'Autumn'}
        return season_month_north.get(month)
    else:
        print('Invalid selection. Please select a hemisphere and try again')


sr_year_estuary = sr[['ID_NUMBER', 'p_ID','year','month', 'Corrected_LS_Temp']]
sr_year_estuary = sr_year_estuary.merge(estuary_shp, on='ID_NUMBER', how='left')
south = sr_year_estuary[sr_year_estuary['INPUT_LAT'] <0]
north = sr_year_estuary[sr_year_estuary['INPUT_LAT'] >0]

season_list = []
hemisphere = 'Southern'
for month in south['month']:
    season = find_season(month, hemisphere)
    season_list.append(season)
    
south['Season'] = season_list

season_list = []
hemisphere = 'Southern'
for month in north['month']:
    season = find_season(month, hemisphere)
    season_list.append(season)
    
north['Season'] = season_list

sr_year_estuary = pd.concat([north, south])

a = sr_year_estuary.groupby(['ID_NUMBER', 'year','Season'])['Corrected_LS_Temp'].size().reset_index()
a['key'] = a['ID_NUMBER'].astype(str) +"_"+a['Season']
a = a.merge(estuary_shp, on='ID_NUMBER', how='left')
b = a.groupby(['ID_NUMBER','Season'])['Corrected_LS_Temp'].max().reset_index()
b['key'] = b['ID_NUMBER'].astype(str) +"_"+b['Season']

c = a.merge(b, on='key', how='left')
c['percentage'] = c['Corrected_LS_Temp_x']/c['Corrected_LS_Temp_y']
c = c[c['percentage'] > 0.7]

In [5]:
estuary_shp = geopandas.read_file(r"D:\Estuary Temperature\global sr\final\shp\estuary_centroid.shp")
def find_season(month, hemisphere):
    if hemisphere == 'Southern':
        season_month_south = {
            12:'Summer', 1:'Summer', 2:'Summer',
            3:'Autumn', 4:'Autumn', 5:'Autumn',
            6:'Winter', 7:'Winter', 8:'Winter',
            9:'Spring', 10:'Spring', 11:'Spring'}
        return season_month_south.get(month)
        
    elif hemisphere == 'Northern':
        season_month_north = {
            12:'Winter', 1:'Winter', 2:'Winter',
            3:'Spring', 4:'Spring', 5:'Spring',
            6:'Summer', 7:'Summer', 8:'Summer',
            9:'Autumn', 10:'Autumn', 11:'Autumn'}
        return season_month_north.get(month)
    else:
        print('Invalid selection. Please select a hemisphere and try again')


sr_year_estuary = sr[['ID_NUMBER', 'p_ID','year','month', 'Corrected_LS_Temp']]
sr_year_estuary = sr_year_estuary.merge(estuary_shp, on='ID_NUMBER', how='left')
south = sr_year_estuary[sr_year_estuary['INPUT_LAT'] <0]
north = sr_year_estuary[sr_year_estuary['INPUT_LAT'] >0]

season_list = []
hemisphere = 'Southern'
for month in south['month']:
    season = find_season(month, hemisphere)
    season_list.append(season)
    
south['Season'] = season_list

season_list = []
hemisphere = 'Southern'
for month in north['month']:
    season = find_season(month, hemisphere)
    season_list.append(season)
    
north['Season'] = season_list

sr_year_estuary = pd.concat([north, south])
#### More than 70% of water from above calculation
sr_year_estuary['key'] = sr_year_estuary['ID_NUMBER'].astype(str) +"_"+sr_year_estuary['Season']
sr_year_estuary = sr_year_estuary[sr_year_estuary['key'].isin(c.key)]

##### More than 3 seasons

season = sr_year_estuary[['ID_NUMBER', 'p_ID','year','month','Season', 'Corrected_LS_Temp']].groupby(['ID_NUMBER',  'year','Season'])['Corrected_LS_Temp'].mean().reset_index()
season =season.groupby(['ID_NUMBER','year']).size().reset_index(name='counts')
season = season[season.counts >3]
season['key'] = season['ID_NUMBER'].astype(str) +"_"+ season['year'].astype(str) 

year = sr_year_estuary.groupby(['ID_NUMBER', 'p_ID', 'year','Season'])['Corrected_LS_Temp'].mean().reset_index()
year = year.groupby(['ID_NUMBER', 'year','Season'])['Corrected_LS_Temp'].mean().reset_index()

year = year.groupby(['ID_NUMBER', 'year'])['Corrected_LS_Temp'].mean().reset_index()
year['key'] = year['ID_NUMBER'].astype(str) +"_"+ year['year'].astype(str) 
year=year[year['key'].isin(season['key'])]

#More than 10 years of observation
count_years = year.groupby(['ID_NUMBER']).size().reset_index(name='counts')
count_years = count_years[count_years.counts > 10]
year = year[year['ID_NUMBER'].isin(count_years['ID_NUMBER'])]


test_each_estuary = year.groupby('ID_NUMBER', as_index=False)['Corrected_LS_Temp'].apply(mktest)

test_each_estuary = test_each_estuary.merge(estuary_shp, on='ID_NUMBER', how='left')

bins = [-60,-30, 0, 30, 60, 90]
num = ['60-30 S ','30-0 S','0-30 N','30-60 N ','60-90 N']
test_each_estuary['Latitude Zone']=pd.cut(test_each_estuary['INPUT_LAT'],bins,labels=num)
seasons4 = test_each_estuary

In [6]:
#More than 2 seasons
season = sr_year_estuary[['ID_NUMBER', 'p_ID','year','month','Season', 'Corrected_LS_Temp']].groupby(['ID_NUMBER',  'year','Season'])['Corrected_LS_Temp'].mean().reset_index()
season =season.groupby(['ID_NUMBER','year']).size().reset_index(name='counts')
season = season[season.counts >2]
season['key'] = season['ID_NUMBER'].astype(str) +"_"+ season['year'].astype(str) 

year = sr_year_estuary.groupby(['ID_NUMBER', 'p_ID', 'year','Season'])['Corrected_LS_Temp'].mean().reset_index()
year = year.groupby(['ID_NUMBER', 'year','Season'])['Corrected_LS_Temp'].mean().reset_index()

year = year.groupby(['ID_NUMBER', 'year'])['Corrected_LS_Temp'].mean().reset_index()
year['key'] = year['ID_NUMBER'].astype(str) +"_"+ year['year'].astype(str) 
year=year[year['key'].isin(season['key'])]

#More than 10 years of observation
count_years = year.groupby(['ID_NUMBER']).size().reset_index(name='counts')
count_years = count_years[count_years.counts > 10]
year = year[year['ID_NUMBER'].isin(count_years['ID_NUMBER'])]


test_each_estuary = year.groupby('ID_NUMBER', as_index=False)['Corrected_LS_Temp'].apply(mktest)

test_each_estuary = test_each_estuary.merge(estuary_shp, on='ID_NUMBER', how='left')

bins = [-60,-30, 0, 30, 60, 90]
num = ['60-30 S ','30-0 S','0-30 N','30-60 N ','60-90 N']
test_each_estuary['Latitude Zone']=pd.cut(test_each_estuary['INPUT_LAT'],bins,labels=num)
season3 = test_each_estuary

In [7]:
print(seasons4.shape)
print(season3.shape)
print(season3[season3.ID_NUMBER.isin(seasons4['ID_NUMBER'])].shape)
print(seasons4.INPUT_LAT.max())
print(season3.INPUT_LAT.max())

(677, 29)
(958, 29)
(677, 29)
60.48271
72.81475


In [8]:
#Combine the seasons
test_each_estuary = pd.concat([seasons4, season3[season3.INPUT_LAT > 60.48271]]).reset_index()
test_each_estuary.to_csv("test_each_estuary_final_V2.csv", index=False)
print(test_each_estuary.shape)
test_each_estuary = test_each_estuary[~(test_each_estuary.trend=='no trend')]
print(test_each_estuary.INPUT_LAT.max())

(737, 30)
69.24593


In [9]:
import geopandas
import pandas as pd
test_each_estuary = pd.read_csv(r"test_each_estuary_final_V2.csv")
test_each_estuary = test_each_estuary[test_each_estuary.trend != "no trend"]
estuary_shp = geopandas.read_file(r"D:\Estuary Temperature\global sr\final\shp\estuary_centroid.shp")

def find_season(month, hemisphere):
    if hemisphere == 'Southern':
        season_month_south = {
            12:'Summer', 1:'Summer', 2:'Summer',
            3:'Autumn', 4:'Autumn', 5:'Autumn',
            6:'Winter', 7:'Winter', 8:'Winter',
            9:'Spring', 10:'Spring', 11:'Spring'}
        return season_month_south.get(month)
        
    elif hemisphere == 'Northern':
        season_month_north = {
            12:'Winter', 1:'Winter', 2:'Winter',
            3:'Spring', 4:'Spring', 5:'Spring',
            6:'Summer', 7:'Summer', 8:'Summer',
            9:'Autumn', 10:'Autumn', 11:'Autumn'}
        return season_month_north.get(month)
    else:
        print('Invalid selection. Please select a hemisphere and try again')

air_temp = pd.read_csv(r"D:\GDrive\er5_weight_scale100.csv")
air_temp= air_temp.dropna(subset=['mean'])
air_temp['mean_2m_air_temperature'] = air_temp['mean']

air_temp ['year'] = air_temp ['system:index'].str.slice(start=0, stop=4).astype(int)
air_temp['month'] = air_temp['system:index'].str.slice(start=4, stop=6).astype(int)
air_temp = air_temp[['year', 'month', 'mean_2m_air_temperature','ID_NUMBER']].dropna()
air_temp = air_temp[(air_temp.year < 2021) & (air_temp.year > 1984)]
air_temp ['mean_2m_air_temperature'] = air_temp ['mean_2m_air_temperature'] - 273.15
air_temp

air_temp= air_temp.merge(estuary_shp, on='ID_NUMBER', how='left')
south = air_temp[air_temp['INPUT_LAT'] <0]
north = air_temp[air_temp['INPUT_LAT'] >0]

season_list = []
hemisphere = 'Southern'
for month in south['month']:
    season = find_season(month, hemisphere)
    season_list.append(season)
    
south['Season'] = season_list

season_list = []
hemisphere = 'Southern'
for month in north['month']:
    season = find_season(month, hemisphere)
    season_list.append(season)
    
north['Season'] = season_list

air_temp = pd.concat([north, south])

air_temp = air_temp.groupby([ 'ID_NUMBER','year', 'Season'])['mean_2m_air_temperature'].mean().reset_index()
air_temp = air_temp[['ID_NUMBER', 'year', 'mean_2m_air_temperature']].groupby([ 'ID_NUMBER','year'])['mean_2m_air_temperature'].mean().reset_index()


air_temp_test  = air_temp.groupby('ID_NUMBER', as_index=False)['mean_2m_air_temperature'].apply(mktest)
air_temp_test = air_temp_test[air_temp_test.ID_NUMBER.isin(test_each_estuary.ID_NUMBER)]
air_temp_test.groupby('trend').size()
air_temp_test.to_csv("air_temp_test.csv", index=False)